## Time Series
Columns
* Sitename
* Range
* Column
* Date
* Canopy Height
* Cumulative GDD
* Environmental Factors
    * Min / Max / Mean Temp
    * Humidity
    * others. . . 

In [1]:
import datetime
import numpy as np
import pandas as pd
import sqlalchemy
import sqlite3

In [5]:
df_0 = pd.read_csv('data/raw/mac_season_4.csv', low_memory=False)
df_0.shape

(372363, 39)

In [ ]:
# df_0.columns

In [6]:
cols_to_drop = ['Unnamed: 0', 'checked', 'result_type', 'id', 'citation_id', 'treatment_id', 'city', 
                'scientificname', 'commonname', 'genus', 'species_id', 'author', 'citation_year', 'treatment', 
                'time', 'raw_date', 'month', 'year', 'dateloc', 'trait_description', 'units', 'n', 'statname',
                'stat', 'notes', 'access_level', 'entity', 'method_name', 'view_url', 'edit_url']

In [7]:
df_1 = df_0.drop(labels=cols_to_drop, axis=1)
# df_1.head()

In [9]:
# df_1.shape

In [ ]:
# print(df_1.date.min())
# print(df_1.date.max())

In [10]:
# Do all dates in the df contain the string (America/Phoenix)?

count = 0
for d in df_1.date.values:
    if 'America/Phoenix' in d:
        count += 1

count

16482

In [11]:
# Do all the dates start with 2017?

count = 0
for d in df_1.date.values:
    if d.startswith('2017'):
        count += 1
        
count

372363

### I. Change date values to iso date format
* date(s) as index?

In [12]:
new_dates = []

for d in df_1.date.values:
    
    if 'Phoenix' in d:
        new_name = d[:-18]
        new_dates.append(new_name)
    
    else:
        new_name = d
        new_dates.append(new_name)
        
print(len(new_dates))

372363


In [13]:
iso_format_dates = pd.to_datetime(new_dates)

In [14]:
df_1['date_1'] = iso_format_dates
# df_1.head()

### II. Change sitenames to strip E and W subplot designations
* Check which traits are associated with subplots

In [16]:
e_w_plots = df_1.loc[(df_1.sitename.str.endswith(' E')) | (df_1.sitename.str.endswith(' W'))]
e_w_plots.trait.unique()

array(['leaf_temperature', 'planter_seed_drop', 'seedling_emergence_rate',
       'panicle_height', 'stand_count', 'canopy_height', 'leaf_length',
       'leaf_width', 'canopy_cover', 'stem_elongated_internodes_number',
       'emergence_count', 'stalk_diameter_fixed_height',
       'stalk_diameter_minor_axis', 'stalk_diameter_major_axis',
       'plant_basal_tiller_number'], dtype=object)

In [17]:
e_w_plot_traits = e_w_plots.trait.unique()

In [18]:
sitename_values = df_1.sitename.values
no_e_w_names = []

for name in sitename_values:
    
    if name.endswith(' W') | name.endswith(' E'):
        name = name[:-2]
        no_e_w_names.append(name)
        
    else:
        no_e_w_names.append(name)

print(len(no_e_w_names))

372363


In [19]:
df_1['sitename_1'] = no_e_w_names
# df_1.sample(n=7)

In [ ]:
# df_1.loc[df_1.sitename.str.endswith(' E')].iloc[0]

#### III. Extract Range and Column Values

In [20]:
df_1['range'] = df_1['sitename'].str.extract("Range (\d+)").astype(int)
df_1['column'] = df_1['sitename'].str.extract("Column (\d+)").astype(int)

# df_1.sample(n=7)

#### Drop duplicated Columns, Reorder, Rename, & Sort by Date
* `site_id` will be dropped because of different ids associated with subplots

In [22]:
df_2 = df_1.drop(labels=['site_id', 'sitename', 'date'], axis=1)
df_2.shape

(372363, 10)

In [23]:
df_2.columns

Index(['lat', 'lon', 'cultivar_id', 'trait', 'mean', 'cultivar', 'date_1',
       'sitename_1', 'range', 'column'],
      dtype='object')

In [24]:
col_reorder = ['date_1', 'sitename_1', 'range', 'column', 'lat', 'lon', 'cultivar', 'cultivar_id', 'trait', 'mean']

In [25]:
df_3 = pd.DataFrame(data=df_2, columns=col_reorder, index=df_2.index)
# df_3.head()

In [26]:
df_4 = df_3.rename({'date_1': 'date', 'sitename_1': 'sitename', 'mean': 'value'}, axis=1)
# df_4.tail()

In [28]:
df_5 = df_4.sort_values(by=['date'], ascending=True)
print(df_5.shape)
print(df_4.shape)
# df_5.head()

(372363, 10)
(372363, 10)


In [ ]:
# df_5.tail()

### III. Drop traits not needed at this time
* dropping `canopy_height` because of the E W subplots - dealing with those values separately

In [30]:
# df_5.trait.unique()

In [31]:
traits_to_keep = ['flowering_time', 'flag_leaf_emergence_time', 'light_intensity_PAR', 'ambient_humidity', 
                  'proximal_air_temperature', 'aboveground_biomass_moisture', 'aboveground_dry_biomass', 
                  'aboveground_fresh_biomass']

In [32]:
df_6 = df_5.loc[df_5.trait.isin(traits_to_keep)]
df_6.shape

(7438, 10)

In [33]:
df_6.isnull().sum()

date           0
sitename       0
range          0
column         0
lat            0
lon            0
cultivar       0
cultivar_id    0
trait          0
value          0
dtype: int64

In [37]:
# df_6.head()

Double check that there are no E W subplots in this smaller dataset.

In [35]:
e_w_plot_traits

array(['leaf_temperature', 'planter_seed_drop', 'seedling_emergence_rate',
       'panicle_height', 'stand_count', 'canopy_height', 'leaf_length',
       'leaf_width', 'canopy_cover', 'stem_elongated_internodes_number',
       'emergence_count', 'stalk_diameter_fixed_height',
       'stalk_diameter_minor_axis', 'stalk_diameter_major_axis',
       'plant_basal_tiller_number'], dtype=object)

Traits in `traits_to_keep` that also had E W subplot designations

In [36]:
set(e_w_plot_traits) & set(traits_to_keep)

set()

### Add TO Mappings

#### Final Steps
* Create `.csv`

In [38]:
need_to_create_csv = True

if need_to_create_csv:

    timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
    output_filename = f'tall_format_traits_{timestamp}.csv'.replace(':', '')
    df_6.to_csv(f'data/processed/{output_filename}')

## Not Using Currently

### Add Cumulative GDD
Note: this data is from the MAC Weather Station
* compare / use new geostreams data in weather data notebook

In [ ]:
# gdd_df = pd.read_csv('data/processed/daily_temps_gdd_2019-12-16T120237.csv')
# gdd_df.head()

In [ ]:
# Check for any days that would have a daily value of less than zero

# daily_value_list = ((gdd_df.air_temp_min + gdd_df.air_temp_max) / 2) - 10
# print(len(gdd_df))
# print(len(daily_value_list))

In [ ]:
# should have no output

# for i in daily_value_list:
    
#     if i < 0:
#         print(i)

In [ ]:
# date_list = gdd_df.date.values
# gdd_list = gdd_df.gdd.values

# date_gdd_dict = dict(zip(date_list, gdd_list))
# date_gdd_dict